# 1. Understanding The Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('../input/bank-marketing/bank-additional-full.csv', sep = ';')

In [ ]:
print(dataset.shape)

In [ ]:
print(dataset.head())

In [ ]:
print(dataset.info())

There are no null values in the dataset. On the other hand, there are some categorical variables.

# 2. Cleaning The Dataset

In [ ]:
dataset = dataset.rename(columns={'y': 'subscribed'})

I will check duplicated values.

In [ ]:
print(dataset.duplicated().sum())

There are 12 duplicated values in the dataset. Let's see which data points are duplicated.

In [ ]:
print(dataset[dataset.duplicated(keep=False)].iloc[:,:7])

I will drop duplicated values from the dataset.

In [ ]:
dataset = dataset.drop_duplicates()

In [ ]:
print(dataset.shape)

In [ ]:
print('\033[1mNULL VALUES\033[0m\n'+ str(dataset.isnull().values.any()))

There are no null values in the dataset.

# 3. Exploratory Data Analysis

There is a graphic below who subscribe a term deposit or not.

In [ ]:
Subscribed = pd.DataFrame(dataset['subscribed'].value_counts())
print(Subscribed.T)
pd.DataFrame(dataset['subscribed'].value_counts()).plot(kind='bar', color='lightgreen')
plt.show()

Most people do not subscribe to a term deposit.

In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(1,4,1)
sns.distplot(dataset['age'])
plt.title('Age Distribution')

plt.subplot(1,4,2)
sns.countplot(dataset['job'])
plt.title('Job Distribution')
plt.xticks(rotation=90)

plt.subplot(1,4,3)
sns.countplot(dataset['marital'], color='pink')
plt.title('Marital Status')

plt.subplot(1,4,4)
sns.countplot(dataset['education'], color='lightgreen')
plt.xticks(rotation=90)
plt.title('Education Level')

plt.show()

* Most people are generally between 20 - 40 years old. Few people are above 60 years old. 
* Most people are administrator, technician or blue-collar workers.
* Most people are married.
* Most people have university degree.

In [ ]:
plt.figure(figsize=(16,5))

plt.subplot(1,3,1)
sns.countplot(dataset['default'], palette="Set3")
plt.title('Default Credit')

plt.subplot(1,3,2)
sns.countplot(dataset['housing'], palette="Set3")
plt.title('Housing Loan')

plt.subplot(1,3,3)
sns.countplot(dataset['loan'], palette="Set3")
plt.title('Loan')

plt.show()

* Most people have no credit in default, while almost none of people have credit.
* The number of people who have housing loan are higher than people who have no housing loan.
* Most people have no personal loan. 

In [ ]:
plt.figure(figsize=(18,5))

plt.subplot(1,4,1)
sns.countplot(dataset['contact'], palette="vlag")
plt.title('Contact Type')

plt.subplot(1,4,2)
sns.countplot(dataset['month'], palette="vlag",order = ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])
plt.title('Month')
plt.xticks(rotation=90)

plt.subplot(1,4,3)
sns.countplot(dataset['day_of_week'], palette="vlag")
plt.title('Day Of Week')

plt.subplot(1,4,4)
sns.distplot(dataset['duration'])
plt.xticks(rotation=90)
plt.title('Duration of Calls')

plt.show()

* Customers were contacted almost everyday. It doesn't convey extra infrmation, that's why I will 'drop day_of_week' from the dataset.

* Most people are reached from cellular phones.
* In may, most calls are made.
* Duration of calls are generally between 0 - 1000 sn.

In [ ]:
dataset.drop('day_of_week', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,5))

plt.subplot(1,2,1)
sns.violinplot("contact", "campaign", data=dataset, kind='reg')
plt.title('Number of Contacts vs Contact Type')

plt.subplot(1,2,2)
sns.distplot(dataset['campaign'])
plt.title('Number of Contacts with Customers')

plt.show()

The number of contacts performed during this campaign and for this client are higher with telephone. On the other hand, the number of contacts is around 0-10 range.

In [ ]:
plt.figure(figsize=(16,5))

plt.subplot(1,3,1)
sns.countplot(dataset['pdays'])
plt.xticks(rotation=90)
plt.title('Number of Days Passed Since Previous Campaign')

plt.subplot(1,3,2)
sns.countplot(dataset['previous'])
plt.title('Number of Previous Contacts')

plt.subplot(1,3,3)
sns.countplot(dataset['poutcome'])
plt.title('Previous Campaign Result')

plt.show()

* "pdays" show the number of days that passed by after the client was last contacted from a previous campaign (999 means client was not previously contacted). Graphic tells us that almost all of the customers were not contacted for previous campaign. I will change this variable as previously contacted or not, since most of the clients were not previously contacted. 

* The number of previous contacts graphic shows that most people were not contacted previously. That's why previous campaign results do not exist for some customers.

In [ ]:
dataset.loc[dataset['pdays'] < 999, 'pdays'] = 1
dataset.loc[dataset['pdays'] == 999, 'pdays'] = 0

In [ ]:
dataset = dataset.rename(columns={'pdays': 'previouslycontacted', 'previous':'previouscontacts'})

# 4. Data Preparation

I will change ages as age group and I will drop age column from the dataset.

In [ ]:
bins= [0,10,20,30,40,50,60,70,80,90,100]
labels = [0,1,2,3,4,5,6,7,8,9]
dataset.insert(1, 'agegroup', pd.cut(dataset['age'], bins=bins, labels=labels, right=False))
dataset = dataset.drop('age', axis=1)

### Converting to numerical values

I will convert categorical variables to numerical variables with Label Encoder from Sklearn.

In [ ]:
categorical_columns = dataset.select_dtypes(include='object').columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in categorical_columns:
    dataset[i] = le.fit_transform(dataset[i]) 

In [ ]:
print(dataset.head())

In [ ]:
print(dataset.shape)

### Checking Normal Distribution

Gaussian Naive Bayes assumes that the predictors take up a continuous value and are not discrete, and these values are sampled from a gaussian distribution. That's why I will check normal distribution.

In [ ]:
X_train = dataset.iloc[:, :-1].values.astype('float')
y_train = dataset['subscribed'].values

In [ ]:
pd.DataFrame(X_train[y_train == 1]).plot(kind='density', ind=100, legend=False)
plt.title('Subscribed Likelihood Plots')

plt.show()

In [ ]:
pd.DataFrame(X_train[y_train == 0]).plot(kind='density', ind=100, legend=False)
plt.title('Not Subscribed Likelihood Plots')

plt.show()

The data points are not normally distributed. I will apply Standard Scaler to get a more normally distributed dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))

In [ ]:
X_train[y_train == 1].plot(kind='density', ind=100, legend=False)
plt.title('Subscribed Likelihood Plot after Standardization')
plt.show()

In [ ]:
X_train[y_train == 0].plot(kind='density', ind=100, legend=False)
plt.title('Not Subscribed Likelihood Plot after Standardization')
plt.show()

There are values that go as 1,2,3 ... These values will not have perfectly normal distribution even after standardization. However, it gives better distribution.

### Checking Correlation Between Features

The correlation between variables are not linear, that's why I will use spearman correlation method.

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(dataset.corr(method='spearman'), cbar=True, cmap="RdBu_r")
plt.title("Correlation Matrix", fontsize=16)
plt.show()

There are some highly correlated variables in the dataset. Since Naive Bayes assumes that features are independent of each other, I will drop highly correlated varibles.

In [ ]:
correlation = X_train.corr(method='spearman').abs()
upper = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.40)]
X_train.drop(X_train[to_drop], axis=1, inplace=True)

In [ ]:
print(X_train.shape)

# 5. Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.25, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gb = GaussianNB()
gb.fit(X_train, y_train)
pred = gb.predict(pd.DataFrame(X_test))

# 6. Evaluation

In [ ]:
from sklearn.metrics import roc_curve, auc
gbprob = gb.predict_proba(X_train)[:,1]
fpr, tpr, thr = roc_curve(y_train, gbprob)
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Plot')
print(auc(fpr, tpr))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print('Accuracy score of Gaussian Naive Bayes:' + str(accuracy_score(y_test,pred)))
print('Confusion Matrix\n' + str(confusion_matrix(y_test, pred)))

The model predicted 383 false positives. It means it predicted as the 383 customers subscribe to a term deposit, but actually it is not true. These customers don't subscribe. This number is actually high. The model should not miss this, othervise they do not call the customer again, and lose the customer.